##### 0-1 KNAPSACK PROBLEM


For this assignment, i have tried to design an algorithm that search the best configuration of items in a set of knapsack, such that we have the maximum possible values and all the restraint are satisfied

To do that, I used a hill climbing method, starting from a greedy starting point, finding a good starting point that is also feasible with a random greed algorithm. The greed algorithm was an idea taken from Riccardo.
The algorithm starts by creating a feasible greedy solution from a set of empty knapsacks that satisfy all the constraints (if i can add an item to its ideal knapsack i do it). Then, I iteratively tweak this solution changing the status of an item a random number of time (decided with a threshold that diminuish every iteration).
During the process i save the best solution but also a current solution. I use an simulated annealing to change the current solution.



After testing the problems more times, the  best results has been:
| **Test** | **Best Value** | **Time** | 
|:---------:|:-----------:|:----------:|
| 1 | 1,065 | 0.2 s | 
| 2 | 45890 | 5 s | 

For problem 3 things get difficult. The items are too much to do a complete greedy initialitation feasible. The best is to find a good compromise between the base solution and the tweak implementation. The best value reached is 1400k.


In [1]:
import numpy as np
import random
import time
import math


In [2]:
def FailureOfSolution(solution,WEIGHTS,CONSTRAINTS):
    #an item can be in only one knapsack
    if not np.all(solution.sum(axis=0) <= 1):
        return False
    if not np.all(solution @ WEIGHTS <= CONSTRAINTS):
        return False
    return True

In [44]:
def BuiltASolution(NUM_KNAPSACKS, NUM_ITEMS, WEIGHTS, CONSTRAINTS, VALUES, max_items=500):



    solution = np.full((NUM_KNAPSACKS, NUM_ITEMS), False)
    new_solution = solution.copy()
    remaining = np.array(CONSTRAINTS, dtype=float)

    # compute ratio = value / total weight across dimensions to avoid broadcasting issues
    weights_sum = np.sum(WEIGHTS, axis=1)
    # avoid division by zero: replace zeros with a very small number
    weights_sum_safe = np.where(weights_sum == 0, 1e-9, weights_sum)
    ratio = np.array(VALUES) / weights_sum_safe
    items = np.argsort(-ratio)
    # if user requests only the first M items, limit items list
    if max_items is not None:
        M = min(int(max_items), NUM_ITEMS)
        items = items[:M]
    #items = rng.permutation(NUM_ITEMS)


    for i in items:
        best_knapsack = -1
        best_residual = float('inf')

        for k in range(NUM_KNAPSACKS):
            # check that item fits in all dimensions of knapsack k
            if np.all(WEIGHTS[i] <= remaining[k]):
                residual = remaining[k] - WEIGHTS[i]
                # use a scalar score (sum of residuals) to compare knapsacks
                residual_score = np.sum(residual)
                if residual_score < best_residual:
                    best_residual = residual_score
                    best_knapsack = k
        if best_knapsack != -1:
            new_solution[best_knapsack, i] = True
            # accept the new assignment only if the solution remains valid
            if FailureOfSolution(new_solution,WEIGHTS,CONSTRAINTS):
                solution[best_knapsack, i] = True
                remaining[best_knapsack] -= WEIGHTS[i]
            else:
                new_solution[best_knapsack, i] = False
    return solution


In [4]:
def EvaluationOfSolution(solution, VALUES):
    sum=solution.sum(axis=0)
    val=np.dot(sum,VALUES)
    return val

In [5]:
def TweakSolution(solution,WEIGHTS,CONSTRAINTS, VALUES, NUM_ITEMS,NUM_KNAPSACKS, threshold=0.8):
    new_solution=solution.copy()
    while random.random()<threshold:

        i=np.random.randint(0,NUM_ITEMS)
        j=np.random.randint(0,NUM_KNAPSACKS)
        new_solution[j][i]= not new_solution[j][i]
    
    if FailureOfSolution(new_solution,WEIGHTS,CONSTRAINTS):
        return (True, new_solution)
    return (False,solution)

In [58]:
def solvingFunction(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSION, VALUES, WEIGHTS, CONSTRAINTS, T0=0.3, eps=0.9999, steps=100000, rng_seed=None, show_progress=False):
   
    rng = np.random.default_rng(rng_seed)

    # initial solutions: greedy starting points
    # since it is a greedy deterministic function, make no sense to call it twice, it would return the same result
    # this can be also a problem of your starting point, since it is a deterministic one and not random
    best_solution = BuiltASolution(NUM_KNAPSACKS, NUM_ITEMS, WEIGHTS, CONSTRAINTS, VALUES)
    current_solution = best_solution.copy()
    current_value = EvaluationOfSolution(current_solution, VALUES)
    best_value = current_value
    print("strting value:",current_value)

    T = T0
    S=0.8

    for step in range(steps):
        # propose a neighbor by tweaking the current solution
        candidate = current_solution.copy()
        new_flag, candidate = TweakSolution(candidate, WEIGHTS, CONSTRAINTS, VALUES, NUM_ITEMS, NUM_KNAPSACKS, threshold=S)
        # evaluate candidate
        candidate_value = EvaluationOfSolution(candidate, VALUES)
        delta = candidate_value - current_value
        accepted = False
        if delta >= 0:
            accepted = True
        else:
            # accept worse with probability exp(delta / T) (delta < 0)
            if T > 0:
                prob = math.exp(delta / T)
            else:
                prob = 0.0
            if random.random() < prob:
                accepted = True

        if accepted:
            current_solution = candidate
            current_value = candidate_value
            # update best if improved
            if current_value > best_value:
                best_solution = current_solution
                best_value = current_value
                if show_progress:
                    print("New best solution found with value:", best_value)

        # cooling schedule
        T *= eps
        S*=eps

        if show_progress and step % 1000 == 0:
            print(f"step={step} T={T:.6e} current={current_value} best={best_value}")

    return best_solution

In [59]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

solution=solvingFunction(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS,VALUES,WEIGHTS, CONSTRAINTS)

print("Final solution:")
print (EvaluationOfSolution(solution,VALUES)) 

strting value: 1065
Final solution:
1065


In [60]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS,size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

solution=solvingFunction(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS,VALUES,WEIGHTS, CONSTRAINTS)

print("Final solution:")
print (EvaluationOfSolution(solution,VALUES)) 

strting value: 45890
Final solution:
45890


In [ ]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))
solution=solvingFunction(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS,VALUES,WEIGHTS, CONSTRAINTS)

print("Final solution:")
print (EvaluationOfSolution(solution,VALUES)) 